# Week 5 Lab: Supervised Learning

<img align="right" style="padding-right:10px;" src="figures_wk5/knn.png" width=300><br>

This week's assignment will focus completeing a KNN analysis and comparing its performance with other supervised algorithms.


## Our Dataset: 
**Dataset:** bank-additional-full.csv (Provided in folder assign_wk5)

Remember to take a look at the bank-additional-names.txt files for a better understanding of the dataset.


# Assignment Requirements

## Part 1: KNN Analysis
**Objective:** According to the dataset's text file, the target column the last column in the dataset.
   - Cleanup the datset as you see deem appropriate. As always, defend your reasoning!!!
       - Missing values?
       - Column names
   - Prepare the data for machine learning
       - A little EDA goeas a long way
       - Do you need to do anything about data types?
   - KNN analysis
       - What is your objective from the analysis?
       - What is your optimal K? 
       - How about accuracy rate? 
    - Discover any insights from this analysis? 
       - Include numbers/graphs corresponding to your conclusions
       - Discuss ways to improve the performance of your KNN model 
       - Defend and backup your thoughts!!!!!!

## Part 2: Comparison to other supervised algorithm
As we saw in the lecture notebook, algorithm performance varies based on the algorithm used. The lecture demostrated using K-Fold Cross-Validation to compare the performance of several algorithm for the same dataset. 
   - At the end of part 1 you discussed ways to improve the performance of you KNN model. 
       - Implement one of those methods to improve your KNN model performance.
       - Rerun a KNN analysis for your improved dataset
       - Discuss the change in performance from the model in part 1
   - Complete a K-fold cross-validation analysis for your improved model
       - You need to use at less three additional models
       - Discuss the difference in the performance of the 4 algorithms against your improved dataset.

# Deliverables:


Upload your Jupyter Notebook to the corresponding location in WorldClass. 

**Note::** Make sure you have clearly indicated each assignment requirement within your notebook.
   
**Important:** Make sure your provide complete and thorough explanations for all of your analysis. You need to defend your thought processes and reasoning.